# Confirm TensorFlow can see the GPU




In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

# Loading libraries


In [0]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import timeit

# Installing PyDrive


In [0]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get Revelant Folders

In [0]:
fileId = drive.CreateFile({'id': '1SacGLfWETpAZCSElYqPd4vy88_1ducIp'}) 
fileId.GetContentFile('Dataset.zip')  # Save Drive file as a local file

fileId = drive.CreateFile({'id': '14QOpq2qoxU6wJZ3gUj5a9BHTPgr0e38C'}) 
fileId.GetContentFile('Result.zip')  # Save Drive file as a local file

!unzip Dataset.zip -d ./
!unzip Result.zip -d ./

# Get Training data


In [0]:
from google.colab import files
#from keras.backend import one_hot 

# Get images to train
TrainImages = []
for filename in os.listdir('Dataset/Train/'):
    TrainImages.append(img_to_array(load_img('Dataset/Train/'+filename)))
TrainImages = np.array(TrainImages, dtype=float)


TrainImages = 1.0/255*TrainImages

Xtrain = rgb2lab(TrainImages)[:,:,:,0] #grayscale
Ytrain = rgb2lab(TrainImages)[:,:,:,1] #ab colorspace

#Z = one_hot(Ytrain)

# CNN architecture

In [0]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(313, (3, 3), activation='relu', padding='same'))

'''model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))'''

model.add(Conv2D(2, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(UpSampling2D((2, 2)))

model.compile(optimizer='rmsprop', loss='mse')
model.summary()

In [0]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='softmax', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [0]:
bias = False

model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))

#Conv1
model.add(Conv2D(filters = 64, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 64, kernel_size = 3, strides=2, activation='relu', padding='same',use_bias=bias))
model.add(BatchNormalization())
#model.add(UpSampling2D((2, 2)))

#Conv2
model.add(Conv2D(filters = 128, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 128, kernel_size = 3, strides=2, activation='relu', padding='same',use_bias=bias))
model.add(BatchNormalization())
#model.add(UpSampling2D((2, 2)))

#Conv3
model.add(Conv2D(filters = 256, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 256, kernel_size = 3, strides=2, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 256, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(BatchNormalization())
#model.add(UpSampling2D((2, 2)))

#Conv4
model.add(Conv2D(filters = 512, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 512, kernel_size = 3, strides=2, activation='relu', padding='same',use_bias=bias))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(filters = 512, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(BatchNormalization())
#model.add(UpSampling2D((2, 2)))

#Conv5
model.add(Conv2D(filters = 512, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias,dilation_rate=1))
model.add(Conv2D(filters = 512, kernel_size = 3, strides=2, activation='relu', padding='same',use_bias=bias,dilation_rate=1))
model.add(Conv2D(filters = 512, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias,dilation_rate=1))
model.add(BatchNormalization())
model.add(UpSampling2D((2, 2)))

#Conv6
model.add(Conv2D(filters = 512, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias,dilation_rate=1))
model.add(Conv2D(filters = 512, kernel_size = 3, strides=2, activation='relu', padding='same',use_bias=bias,dilation_rate=1))
model.add(Conv2D(filters = 512, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias,dilation_rate=1))
model.add(BatchNormalization())
model.add(UpSampling2D((2, 2)))

#Conv7
model.add(Conv2D(filters = 512, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 512, kernel_size = 3, strides=2, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 512, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(BatchNormalization())
model.add(UpSampling2D((2, 2)))

#Conv8
#model8up 
model.add(Conv2DTranspose(filters = 256, kernel_size = 4, strides=2, activation='relu', padding='same',use_bias=bias))

model.add(Conv2D(filters = 256, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 256, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 256, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(BatchNormalization())
#model.add(UpSampling2D((2, 2)))

#model.add(Conv2D(filters = 313,kernel_size=1,activation='softmax'))

#Conv9
model.add(Conv2DTranspose(filters = 128, kernel_size = 4, strides=2, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 128, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 128, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 128, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(BatchNormalization())

#Conv10
model.add(Conv2DTranspose(filters = 128, kernel_size = 4, strides=2, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 128, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 128, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(Conv2D(filters = 128, kernel_size = 3, strides=1, activation='relu', padding='same',use_bias=bias))
model.add(BatchNormalization())

model.add(Conv2D(filters = 2, kernel_size = 3, strides=1, activation='softmax', padding='same',use_bias=False))
#model.add(UpSampling2D((2, 2)))

#model.add(Activation('softmax')) #will give Z hat
#model.add(Activation('softmax'))
#Adam
#rmsprop
model.compile(optimizer='rmsprop', loss='mse')

# Probability Distributions

In [0]:
# Z = H^-1(Ytrain) 
# loss on Z_ and Z
# Y_ = H(z_)

model.summary()



# Training our model


In [0]:
# Image transformer
#Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches).

datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 30

def image_a_b_gen(batch_size):
    for batch in datagen.flow(TrainImages, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0] #grayscale
        Y_batch = lab_batch[:,:,:,1:] / 128 #ab colorsapce /128 to normalize
        yield (X_batch.reshape(X_batch.shape + (1,) ), Y_batch)

# Train model  
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=50, steps_per_epoch=10,shuffle=True)

# Saving our model

In [0]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

# Testing the accuracy of the model

In [0]:
# Test images

TestImages = []
for filename in os.listdir('Dataset/Test/GroundTruth/'):
    if filename == ".DS_Store":
      continue
    else:
      TestImages.append(img_to_array(load_img('Dataset/Test/GroundTruth/'+filename)))
TestImages = np.array(TestImages, dtype=float)


#convert test images to lab colorspace
Xtest = rgb2lab(1.0/255*TestImages)[:,:,:,0] #grayscale version
Xtest = Xtest.reshape(Xtest.shape+(1,))

Ytest = rgb2lab(1.0/255*TestImages)[:,:,:,1:] #ab colorspace
Ytest = Ytest / 128 #normalize

print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

# Color grayscale images

In [0]:
color_me = []
for filename in os.listdir('Dataset/Test/Grayscale/'):
    if filename == ".DS_Store":
      continue
    else:
      color_me.append(img_to_array(load_img('Dataset/Test/Grayscale/'+filename)))

color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    path = "Result/img_"+ str(i)+".jpg"
    imsave(path, lab2rgb(cur))
    